In [1]:
import os

In [2]:
pwd()

'c:\\Users\\mnsnn\\Documents\\Portfolio\\MLOps\\datascienceproject\\research'

In [3]:
os.chdir("../")

In [4]:
pwd()

'c:\\Users\\mnsnn\\Documents\\Portfolio\\MLOps\\datascienceproject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories


class ConfigurationManager:
	def __init__(self, 
		config_filepath = CONFIG_FILE_PATH,
		params_filepath = PARAMS_FILE_PATH,
		schema_filepath = SCHEMA_FILE_PATH):
		
		# Loading configs, params and schema
		self.config = read_yaml(config_filepath)
		self.params = read_yaml(params_filepath)
		self.schema = read_yaml(schema_filepath)

		# Creating Artifacts
		create_directories([self.config.artifacts_root])

	def get_data_ingestion_config(self) -> DataIngestionConfig:
		config = self.config.data_ingestion
		create_directories([config.root_dir])

		data_ingestion_config = DataIngestionConfig(
			root_dir = config.root_dir,
			source_URL = config.source_URL,
			local_data_file = config.local_data_file,
			unzip_dir = config.unzip_dir
		)

		return data_ingestion_config

In [7]:
## Component - Data Ingestion
import requests
import os
import urllib.request as request
from src.datascience import logger
import zipfile

class DataIngestion:
    
	def __init__(self, config:DataIngestionConfig):
		self.config = config

	# Downloading the Zip File
	def download_file(self):
		if not os.path.exists(self.config.local_data_file):
			file_name, headers = request.urlretrieve(
				url = self.config.source_URL,
				filename = self.config.local_data_file
			)
			logger.info(f"{file_name} downloaded with following info: \n{headers}")
		else:
			logger.info(f"File already exists")

	# Extracting the Zip File
	def extract_zip_file(self):
		unzip_path = self.config.unzip_dir
		os.makedirs(unzip_path, exist_ok=True)
		with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
			zip_ref.extractall(unzip_path)


In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-20 17:30:15,197: INFO: common: YAML File: config\config.yaml Loaded Successfully.]
[2025-07-20 17:30:15,201: INFO: common: YAML File: params.yaml Loaded Successfully.]
[2025-07-20 17:30:15,205: INFO: common: YAML File: schema.yaml Loaded Successfully.]
[2025-07-20 17:30:15,208: INFO: common: Created Directory at artifacts]
[2025-07-20 17:30:15,211: INFO: common: Created Directory at artifacts/data_ingestion]
[2025-07-20 17:30:16,629: INFO: 1980136982: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FC5E:1BDC83:1C287C:55F6AB:687CDA53
Accept-Ranges: bytes
Date: Sun, 